<a href="https://github.com/timeseriesAI/tsai-rs" target="_parent"><img src="https://img.shields.io/badge/tsai--rs-Time%20Series%20AI%20in%20Rust-blue" alt="tsai-rs"/></a>

# How to Efficiently Work with Very Large Numpy Arrays

This notebook demonstrates how to work with large numpy arrays that don't fit in memory using **tsai-rs**.

## Introduction

Sometimes datasets are really big (many GBs) and don't fit in memory. The solution is to use **np.memmap** - memory-mapped arrays that work like arrays on disk.

### Key Benefits
- Access small segments of large files without loading entire file into memory
- Fast batch creation for deep learning
- Similar to how image files work - you pass paths, then load on demand

## Install tsai-rs

```bash
cd crates/tsai_python
maturin develop --release
```

## Import Libraries

In [ ]:
import tsai_rs
import numpy as np
from pathlib import Path
import os
import sys

print(f"tsai-rs version: {tsai_rs.version()}")
tsai_rs.my_setup()

## Creating Large Arrays on Disk

For demonstration, we'll create arrays on disk using chunks that fit in memory.

In [ ]:
# Create a data directory
path = Path('./data')
path.mkdir(exist_ok=True)

# Create a moderately large array for demonstration
# In practice, you might have 10GB+ arrays
n_samples = 10000
n_vars = 50
seq_len = 512

print(f"Creating array with shape: ({n_samples}, {n_vars}, {seq_len})")
print(f"Estimated size: {n_samples * n_vars * seq_len * 4 / 1e9:.2f} GB (float32)")

In [ ]:
# Create the array in chunks
def create_large_array_on_disk(path, shape, dtype=np.float32, chunksize=1000):
    """Create a large array on disk in chunks."""
    # Create empty memmap file
    fp = np.memmap(path, dtype=dtype, mode='w+', shape=shape)
    
    # Fill in chunks
    n_samples = shape[0]
    for start in range(0, n_samples, chunksize):
        end = min(start + chunksize, n_samples)
        fp[start:end] = np.random.randn(end - start, *shape[1:]).astype(dtype)
        if start % 5000 == 0:
            print(f"  Progress: {start}/{n_samples}")
    
    fp.flush()  # Ensure data is written to disk
    del fp  # Close the memmap
    print(f"Created array at {path}")

# Uncomment to create the array (takes a few seconds)
# create_large_array_on_disk(path/'X_large.npy', (n_samples, n_vars, seq_len))

## Loading Arrays as Memmaps

The key is to use `mmap_mode` when loading with `np.load`.

In [ ]:
# Create a small demo array
demo_array = np.random.randn(1000, 10, 100).astype(np.float32)
np.save(path/'X_demo.npy', demo_array)

print(f"Saved demo array: {demo_array.shape}")

In [ ]:
# Load as memmap (mode 'c' = copy-on-write)
X_on_disk = np.load(path/'X_demo.npy', mmap_mode='c')

print(f"Loaded memmap: {X_on_disk.shape}")
print(f"Type: {type(X_on_disk)}")

# Key insight: very little RAM is used!
print(f"\nMemory used by memmap object: {sys.getsizeof(X_on_disk)} bytes")
print(f"(Actual data size: {X_on_disk.nbytes} bytes)")

## Memmap Modes

| Mode | Description |
|------|-------------|
| 'r' | Read-only |
| 'r+' | Read and write (changes saved to disk) |
| 'w+' | Create/overwrite for read/write |
| 'c' | Copy-on-write (changes in memory only) |

In [ ]:
# Mode 'c' allows modifications in memory without changing disk
# This is ideal for data augmentation during training

sample = X_on_disk[0].copy()  # Get a sample
sample_modified = sample + 1  # Modify in memory

print("Original disk data unchanged:")
print(f"  X_on_disk[0, 0, 0] = {X_on_disk[0, 0, 0]:.4f}")
print(f"  Modified sample[0, 0] = {sample_modified[0, 0]:.4f}")

## Efficient Slicing

In [ ]:
import time

# Slicing is very fast
def benchmark_slicing(arr, n_trials=1000):
    """Benchmark slicing speed."""
    start = time.time()
    for i in range(n_trials):
        _ = arr[i % len(arr)]
    elapsed = time.time() - start
    return elapsed / n_trials * 1000  # ms per slice

time_per_slice = benchmark_slicing(X_on_disk)
print(f"Time per slice: {time_per_slice:.4f} ms")
print(f"Slices per second: {1000/time_per_slice:.0f}")

In [ ]:
# Slicing along different axes
print("Slicing examples:")
print(f"  X[0] shape: {X_on_disk[0].shape}")  # One sample
print(f"  X[:, 0] shape: {X_on_disk[:, 0].shape}")  # One variable
print(f"  X[:, :, 0] shape: {X_on_disk[:, :, 0].shape}")  # One time step
print(f"  X[:10] shape: {X_on_disk[:10].shape}")  # Batch of 10

## Using with tsai-rs

In [ ]:
# Create labels
n_classes = 5
y = np.random.randint(0, n_classes, size=len(X_on_disk))

# Standardize the memmap (works on slices)
# For very large arrays, you might standardize batch by batch
X_std = tsai_rs.ts_standardize(X_on_disk[:100].astype(np.float32), by_sample=True)

print(f"Standardized batch shape: {X_std.shape}")
print(f"Mean: {X_std.mean():.4f}, Std: {X_std.std():.4f}")

In [ ]:
# Create TSDataset from a batch
batch_size = 64
X_batch = X_on_disk[:batch_size].astype(np.float32)
y_batch = y[:batch_size]

# Standardize
X_batch_std = tsai_rs.ts_standardize(X_batch, by_sample=True)

# Create dataset
ds = tsai_rs.TSDataset(X_batch_std, y_batch)
print(f"Created dataset: {ds}")

## Batch Generator Pattern

In [ ]:
def batch_generator(X_memmap, y, batch_size=32, shuffle=True):
    """Generate batches from memmap array."""
    n_samples = len(X_memmap)
    indices = np.arange(n_samples)
    
    if shuffle:
        np.random.shuffle(indices)
    
    for start in range(0, n_samples, batch_size):
        end = min(start + batch_size, n_samples)
        batch_idx = indices[start:end]
        
        # Load batch from memmap
        X_batch = np.array([X_memmap[i] for i in batch_idx]).astype(np.float32)
        y_batch = y[batch_idx]
        
        # Standardize
        X_batch = tsai_rs.ts_standardize(X_batch, by_sample=True)
        
        yield X_batch, y_batch

# Example usage
for i, (X_batch, y_batch) in enumerate(batch_generator(X_on_disk, y, batch_size=32)):
    if i == 0:
        print(f"Batch 0: X shape = {X_batch.shape}, y shape = {y_batch.shape}")
    if i >= 2:
        break
print(f"Generated {i+1} batches")

## Memory-Efficient Training Pipeline

In [ ]:
def memory_efficient_pipeline(X_path, y, n_epochs=2, batch_size=32):
    """Complete memory-efficient training pipeline."""
    
    # Load as memmap
    X_memmap = np.load(X_path, mmap_mode='c')
    print(f"Loaded memmap: {X_memmap.shape}")
    print(f"RAM used by memmap: ~{sys.getsizeof(X_memmap)} bytes")
    
    n_vars = X_memmap.shape[1]
    seq_len = X_memmap.shape[2]
    n_classes = len(np.unique(y))
    
    # Configure model
    config = tsai_rs.InceptionTimePlusConfig(
        n_vars=n_vars,
        seq_len=seq_len,
        n_classes=n_classes
    )
    print(f"Model config: {config}")
    
    # Training loop
    n_batches_per_epoch = len(X_memmap) // batch_size
    
    for epoch in range(n_epochs):
        print(f"\nEpoch {epoch+1}/{n_epochs}")
        
        for i, (X_batch, y_batch) in enumerate(batch_generator(X_memmap, y, batch_size)):
            # Here you would do actual training
            if i % 10 == 0:
                print(f"  Batch {i}/{n_batches_per_epoch}")
            if i >= 20:  # Demo limit
                break
    
    return config

# Run pipeline
config = memory_efficient_pipeline(path/'X_demo.npy', y, n_epochs=2)

## Cleanup

In [ ]:
# Clean up demo files
import shutil

if (path/'X_demo.npy').exists():
    os.remove(path/'X_demo.npy')
    print("Cleaned up demo files.")

## Summary

### Key Points

1. **Use np.memmap for large arrays**: Allows working with datasets larger than RAM
2. **Mode 'c' for training**: Copy-on-write allows modifications without changing disk data
3. **Batch processing**: Load and process data in chunks
4. **Fast slicing**: Memmap slicing is nearly as fast as in-memory arrays

### Pattern
```python
# Save array
np.save('data.npy', X)

# Load as memmap
X_memmap = np.load('data.npy', mmap_mode='c')

# Use in batches
for batch in batch_generator(X_memmap, y):
    # Train on batch
    ...
```

In [ ]:
# Quick reference
print("Memory-Efficient Arrays Quick Reference")
print("=" * 50)
print("\n# Save array to disk")
print("np.save('data.npy', X)")
print("\n# Load as memmap (copy-on-write)")
print("X = np.load('data.npy', mmap_mode='c')")
print("\n# Access slices efficiently")
print("batch = X[start:end]")
print("\n# Standardize batch")
print("batch_std = tsai_rs.ts_standardize(batch.astype(np.float32), by_sample=True)")